In [1]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
!pip install fake_useragent

In [2]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек
import time          # Библиотека для времени
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import re

In [3]:
page_link = 'https://www.ticketland.ru/spectacle/'
site_link = 'https://www.ticketland.ru'

In [4]:
def get_page_links(page_num):
    result = []
    response = requests.get(page_link + f'page-{page_num}/', headers={'User-Agent': UserAgent().chrome})
    if not response.ok:
        # если сервер нам отказал, вернем пустой лист для текущей страницы
        return []
    
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    objs = soup.findAll('div', attrs = {'class':'card-search__img'})

    for obj in objs:    
        link = site_link + obj.find('a').attrs['href']
        result.append(link)
    return result

In [5]:
get_page_links(1)

['https://www.ticketland.ru/koncertnye-zaly/moskovskiy-dvorec-molodezhi/nichego-ne-boysya-ya-s-toboy/',
 'https://www.ticketland.ru/teatry/teatr-maska/mamma-mimo-ili-myuzikl-poshel-ne-tak/',
 'https://www.ticketland.ru/doma-kultury/dk-im-zueva/idealnyy-muzh/',
 'https://www.ticketland.ru/teatry/teatr-operetty/anna-karenina/',
 'https://www.ticketland.ru/teatry/teatr-operetty/graf-orlov/',
 'https://www.ticketland.ru/teatry/teatrium-na-serpukhovke-p-r-terezy-durovoy/dva-muzha-po-cene-odnogo/',
 'https://www.ticketland.ru/teatry/teatr-ally-dukhovoy-todes/balet-ally-dukhovoy-todes-tancuem-lyubov/',
 'https://www.ticketland.ru/teatry/teatrium-na-serpukhovke-p-r-terezy-durovoy/tyoscha-s-syurprizom/',
 'https://www.ticketland.ru/teatry/teatr-komedii/zhenikhi-po-obyavleniyu/',
 'https://www.ticketland.ru/teatry/teatr-komedii-na-dostoevskoj/sobache-serdce-teatr-komedii/',
 'https://www.ticketland.ru/teatry/teatr-komedii/pigmalion-teatr-komedii/',
 'https://www.ticketland.ru/teatry/teatr-myuzik

In [6]:
def get_rating_and_text(sp_page_link):
    response = requests.get(sp_page_link, headers={'User-Agent': UserAgent().chrome})
    if not response.ok:
        # если сервер нам отказал, вернем пустой лист для текущей страницы
        return {}
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    
    rating = 0
    rating_span = soup.find('span', attrs = {'class':'text-medium'})
    if rating_span:
        rating = float(rating_span.text)
    else:
        return {}
    
    text_div = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['mts-text'])
    
    html_pattern = re.compile('<.*?>')
    clean_text = re.sub(html_pattern, ' ', str(text_div))
    erid_pattern = re.compile('\s+erid: \w+')
    clean_text = re.sub(erid_pattern, '', clean_text)
    clean_text = clean_text.strip()

    return {'rating':rating, 'description':clean_text}

In [7]:
final_df = pd.DataFrame(columns=['rating', 'description'])

for page_number in range(50):
    print(f'page num= {page_number}')
    # собрали хрефы с текущей страницы
    sp_links = get_page_links(page_number)
    for sp_link in sp_links:
        # иногда с первого раза страничка не парсится
        for i in range(3):
            try:
                # пытаемся собрать спектаклю немного даты
                data_row = get_rating_and_text(sp_link)
                # и закидываем её в таблицу
                if data_row:
                    final_df = pd.concat([final_df, pd.DataFrame([data_row])], ignore_index=True)
                # если всё получилось - выходим из внутреннего цикла
                break
            except:
                # Иначе, пробуем еще несколько раз, пока не закончатся попытки
                print('AHTUNG! parsing once again:', sp_link)
                continue
            time.sleep(0.3)

page num= 0
page num= 1
page num= 2
page num= 3
page num= 4
AHTUNG! parsing once again: https://www.ticketland.ru/drugoe/zerkalo-karlosa-santosa/zerkalo-karlosa-santosa/
AHTUNG! parsing once again: https://www.ticketland.ru/drugoe/zerkalo-karlosa-santosa/zerkalo-karlosa-santosa/
AHTUNG! parsing once again: https://www.ticketland.ru/drugoe/zerkalo-karlosa-santosa/zerkalo-karlosa-santosa/
AHTUNG! parsing once again: https://www.ticketland.ru/teatry/teatr-na-melnikova/vse-bulety-prodany/
AHTUNG! parsing once again: https://www.ticketland.ru/teatry/teatr-na-melnikova/vse-bulety-prodany/
AHTUNG! parsing once again: https://www.ticketland.ru/teatry/teatr-na-melnikova/vse-bulety-prodany/
page num= 5
page num= 6
page num= 7
AHTUNG! parsing once again: https://www.ticketland.ru/teatry/teatr-crave/taboo/
AHTUNG! parsing once again: https://www.ticketland.ru/teatry/teatr-crave/taboo/
AHTUNG! parsing once again: https://www.ticketland.ru/teatry/teatr-crave/taboo/
page num= 8
page num= 9
page num= 

In [10]:
final_df.shape 

(850, 2)

In [11]:
final_df.head()

,rating,description
0,9.6,Мюзикл идет с антрактом. Забронируйте с...
1,9.7,Премьера комедийного мюзикла от продюсеров Mam...
2,8.7,«Идеальный муж» - комедия Оскара Уайльда с лов...
3,9.6,«Анна Каренина» — мировой шедевр на одном дыха...
4,9.1,Мюзикл «Граф Орлов» — Театральный блокбастер! ...


In [38]:
final_df.to_csv('spectacle.csv', index = False)

In [39]:
data = pd.read_csv('spectacle.csv')

In [40]:
data.head()

,rating,description
0,9.6,Мюзикл идет с антрактом. Забронируйте с...
1,9.7,Премьера комедийного мюзикла от продюсеров Mam...
2,8.7,«Идеальный муж» - комедия Оскара Уайльда с лов...
3,9.6,«Анна Каренина» — мировой шедевр на одном дыха...
4,9.1,Мюзикл «Граф Орлов» — Театральный блокбастер! ...
